In [7]:
import requests

url = 'http://airviro.klab.ee/station/csv'

data = '?filter[type]=INDICATOR,filter[cancelSearch]=,filter[stationId]=8,filter[dateFrom]=01.01.2022,filter[dateUntil]=31.12.2022,filter[submitHit]=1,filter[indicatorIds]='

r = requests.get('http://airviro.klab.ee/station/csv?filter[type]=INDICATOR,filter[cancelSearch]=,filter[stationId]=8,filter[dateFrom]=01.01.2022,filter[dateUntil]=31.12.2022,filter[submitHit]=1,filter[indicatorIds]=', allow_redirects=True)
print(r.text)


print(r.status_code)


503
